In [9]:
import pandas as pd
import numpy as np
import chardet

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [10]:
file_path = 'OriginData/2023年降雨量.csv'
# 检测文件编码
with open(file_path, 'rb') as f:
    result = chardet.detect(f.read())

# 使用检测到的编码
encoding = result['encoding']
df1 = pd.read_csv(file_path, encoding=encoding)
df1

,日期,日降雨量
0,2023/1/1,1.747288
1,2023/1/2,3.160397
2,2023/1/3,0.857112
3,2023/1/4,1.711757
4,2023/1/5,1.941543
...,...,...
360,2023/12/27,0.068380
361,2023/12/28,1.119148
362,2023/12/29,0.747931
363,2023/12/30,0.310924


In [11]:
file_path = 'OriginData/2024年降雨量.csv'
# 检测文件编码
with open(file_path, 'rb') as f:
    result = chardet.detect(f.read())

# 使用检测到的编码
encoding = result['encoding']
df2 = pd.read_csv(file_path, encoding=encoding)
df2

,日期,日降雨量
0,2024/1/1,0.714507
1,2024/1/2,0.923781
2,2024/1/3,1.090528
3,2024/1/4,0.062974
4,2024/1/5,0.017759
...,...,...
209,2024/10/1,0.834000
210,2024/10/2,0.000000
211,2024/10/5,0.518000
212,2024/10/6,1.082000


In [12]:
import pandas as pd

# 确保日期列是datetime类型，以便能够正确合并
df1['日期'] = pd.to_datetime(df1['日期'],format='%Y/%m/%d')
df2['日期'] = pd.to_datetime(df2['日期'],format='%Y/%m/%d')

# 合并
merged_df = pd.concat([df1,df2], axis=0)

# 将日期时间列转换为仅包含日期的字符串
merged_df['日期'] = merged_df['日期'].dt.strftime('%Y/%m/%d')

In [13]:
merged_df

,日期,日降雨量
0,2023/01/01,1.747288
1,2023/01/02,3.160397
2,2023/01/03,0.857112
3,2023/01/04,1.711757
4,2023/01/05,1.941543
...,...,...
209,2024/10/01,0.834000
210,2024/10/02,0.000000
211,2024/10/05,0.518000
212,2024/10/06,1.082000


# 线性插值补全剩余数据

In [14]:
# 确保时间列是datetime类型
merged_df['日期'] = pd.to_datetime(merged_df['日期'])

# 首先将'日期'列设置为索引
merged_df.set_index('日期', inplace=True)

# 确保时间序列是完整的，使用resample方法来按所需的时间频率填充缺失的时间点
df_resampled = merged_df.resample('D').mean()

# 使用线性插值来填充缺失的数据
df_interpolated = df_resampled.interpolate(method='linear')

# 索引重置为列
df_interpolated.reset_index(inplace=True)

In [15]:
df_interpolated

,日期,日降雨量
0,2023-01-01,1.747288
1,2023-01-02,3.160397
2,2023-01-03,0.857112
3,2023-01-04,1.711757
4,2023-01-05,1.941543
...,...,...
641,2024-10-03,0.172667
642,2024-10-04,0.345333
643,2024-10-05,0.518000
644,2024-10-06,1.082000


In [16]:
df_interpolated.to_csv('OutputData/降雨量总体数据.csv',index=False)